In [2]:
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import re
import nltk

# Parsing

In [3]:
gramatica = """
FRASE -> SUJEITO PREDICADO
SUJEITO -> ENTIDADE
PREDICADO -> V OBJETO
OBJETO -> ENTIDADE
ENTIDADE -> ART N
ART -> 'o' | 'a'
N -> 'lobo'
N -> 'casa'
V -> 'soprou'
"""

In [4]:
from nltk import CFG
from nltk.parse import RecursiveDescentParser

grammar = CFG.fromstring(gramatica)
parser = RecursiveDescentParser(grammar, trace=0)
print(parser.grammar())

Grammar with 10 productions (start state = FRASE)
    FRASE -> SUJEITO PREDICADO
    SUJEITO -> ENTIDADE
    PREDICADO -> V OBJETO
    OBJETO -> ENTIDADE
    ENTIDADE -> ART N
    ART -> 'o'
    ART -> 'a'
    N -> 'lobo'
    N -> 'casa'
    V -> 'soprou'


In [5]:
for p in parser.parse("o lobo soprou a casa".split()):
 #   print(p)
    p.pretty_print()

                  FRASE                         
        ____________|________                    
       |                 PREDICADO              
       |             ________|________           
    SUJEITO         |               OBJETO      
       |            |                 |          
    ENTIDADE        |              ENTIDADE     
  _____|______      |         ________|______    
ART           N     V       ART              N  
 |            |     |        |               |   
 o           lobo soprou     a              casa



# Quem soprou a casa!?

In [6]:
# Pergunta: quem soprou a casa?
trees = []
for t in parser.parse("o lobo soprou a casa".split()):
    trees.append(t)


for subtree in trees[0].subtrees(): # Generate all subtrees
    if subtree.label()=='SUJEITO':
        print(subtree.leaves())

# Pergunta: o que o lobo fez?

# Pergunta: o que o lobo soprou?


['o', 'lobo']


# Modelando ambiguidades

In [7]:
from nltk import PCFG
from nltk.parse import InsideChartParser    


gramatica = """
FRASE -> SUJEITO PREDICADO [1]
SUJEITO -> ENTIDADE [1]
PREDICADO -> V OBJETO [0.5] | V OBJETO ADV [0.5]
OBJETO -> ENTIDADE [0.5] | PREP_ART ENTIDADE [0.5]
ADV -> PREP ENTIDADE [1]
ENTIDADE -> N [0.25] | PROPESS [0.25] | N ADJ [0.5]
ADJ -> PREP N [1]
PROPESS -> 'ele' [1]
V -> 'entrou' [1]
PREP_ART -> 'na' [1]
N -> 'loja' [0.5]
PREP -> 'de' [1]
N -> 'calças' [0.5]
"""

grammar = PCFG.fromstring(gramatica)
parser = InsideChartParser(grammar, trace=0)
print(parser.grammar())

Grammar with 17 productions (start state = FRASE)
    FRASE -> SUJEITO PREDICADO [1.0]
    SUJEITO -> ENTIDADE [1.0]
    PREDICADO -> V OBJETO [0.5]
    PREDICADO -> V OBJETO ADV [0.5]
    OBJETO -> ENTIDADE [0.5]
    OBJETO -> PREP_ART ENTIDADE [0.5]
    ADV -> PREP ENTIDADE [1.0]
    ENTIDADE -> N [0.25]
    ENTIDADE -> PROPESS [0.25]
    ENTIDADE -> N ADJ [0.5]
    ADJ -> PREP N [1.0]
    PROPESS -> 'ele' [1.0]
    V -> 'entrou' [1.0]
    PREP_ART -> 'na' [1.0]
    N -> 'loja' [0.5]
    PREP -> 'de' [1.0]
    N -> 'calças' [0.5]


In [8]:
for p in parser.parse("ele entrou na loja de calças".split()):
    p.pretty_print()
    print("Probability:", p.prob())

                 FRASE                                       
    _______________|_________                                 
   |                     PREDICADO                           
   |        _________________|_______                         
   |       |                       OBJETO                    
   |       |        _________________|_______                 
SUJEITO    |       |                      ENTIDADE           
   |       |       |          _______________|______          
ENTIDADE   |       |         |                     ADJ       
   |       |       |         |                ______|____     
PROPESS    V    PREP_ART     N              PREP         N   
   |       |       |         |               |           |    
  ele    entrou    na       loja             de        calças

Probability: 0.0078125
                 FRASE                                     
    _______________|________________                        
   |                            PREDICADO  

# Chunking (ou: parsing com RegEx)

## Partimos de PoS Tags...

In [9]:
# Carregar o corpus e pré-processar
s = open('./datasets/macmorpho-train.txt', 'r', encoding='utf-8').read()
s = re.split(r'\.+_PU', s)
s = [s0.strip() for s0 in s]
s = [re.split('\s+', s0) for s0 in s]
s = [ [ tuple(re.split('_', w0)) for w0 in p] for p in s]
s = [ [ w for w in p if len(w)==2 ] for p in s ] 
s = [ [ (w[0].lower(), w[1]) for w in p] for p in s ]
s = [p for p in s if len(p)>5]      
# s[frase][palavra] = (palavra, tipo)
#print(s[0:2])

In [10]:
from nltk.tag import NgramTagger, DefaultTagger
taggers = []
taggers.append(DefaultTagger('N'))

for n in range(3):
    taggers.append(NgramTagger(n+1, s, backoff=taggers[-1]))

In [11]:
pos = taggers[-1].tag("o porquinho fugiu para a fazenda".split())
print(pos)

[('o', 'ART'), ('porquinho', 'N'), ('fugiu', 'V'), ('para', 'PREP'), ('a', 'ART'), ('fazenda', 'N')]


In [12]:
gramatica = ('''
    ENTIDADE: {<ART><N>}
    PREDICADO: {<V><PREP>?<ENTIDADE>}
    SUJEITO: {<ENTIDADE>}
    FRASE: {<SUJEITO><PREDICADO>}
    ''')
parser = nltk.RegexpParser(gramatica)
#print(parser)
for p in parser.parse(pos):
    print(p.pretty_print())


                            FRASE                                    
          ____________________|________                               
      SUJEITO                      PREDICADO                         
         |                     ________|______________                
      ENTIDADE                |        |           ENTIDADE          
   ______|__________          |        |        ______|_________      
o/ART          porquinho/N fugiu/V para/PREP a/ART          fazenda/N

None


# Encontrando entidades

In [15]:
pos = taggers[-1].tag("o rio de janeiro continua lindo".split())
print(pos)
gramatica = ('''
    ENTIDADE: {<ART>?<NPROP>}
    ''')
parser = nltk.RegexpParser(gramatica)
#print(parser)
for p in parser.parse(pos):
    print(p)
    #if type(p) != tuple:
    #    print(p.leaves())
    #if type(p)!="<class 'tuple'>" and p.label()=="ENTIDADE":
    #    print (p.leaves())

[('o', 'ART'), ('rio', 'NPROP'), ('de', 'NPROP'), ('janeiro', 'NPROP'), ('continua', 'V'), ('lindo', 'ADJ')]
(ENTIDADE o/ART rio/NPROP)
(ENTIDADE de/NPROP)
(ENTIDADE janeiro/NPROP)
('continua', 'V')
('lindo', 'ADJ')
